In [1]:
import pandas as pd
!pip install geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
!pip install seaborn
import seaborn as sns
!pip install folium
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

(1) Read in the data and examine column information

In [3]:
trash_pickup_data=pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [4]:
trash_pickup_data.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [5]:
trash_pickup_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


(2) Create a dataframe with all missed pickup incidents and change addresses to uniform lowercase

In [7]:
trash_missed_pickups=trash_pickup_data[trash_pickup_data['Request'].str.contains('Miss')]

In [8]:
trash_missed_pickups=trash_missed_pickups.copy()

In [9]:
trash_missed_pickups.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728


In [10]:
trash_missed_pickups.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15028 entries, 1 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    15028 non-null  int64  
 1   Date Opened       15028 non-null  object 
 2   Request           15028 non-null  object 
 3   Description       14999 non-null  object 
 4   Incident Address  15020 non-null  object 
 5   Zip Code          14972 non-null  float64
 6   Trash Hauler      14422 non-null  object 
 7   Trash Route       14405 non-null  object 
 8   Council District  14991 non-null  float64
 9   State Plan X      15007 non-null  float64
 10  State Plan Y      15007 non-null  float64
 11  LONGITUDE         15007 non-null  float64
 12  LATITUDE          15007 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 1.6+ MB


In [11]:
trash_missed_pickups['Address_lower']=trash_missed_pickups['Incident Address'].str.lower()

In [12]:
trash_missed_pickups.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,Address_lower
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470,4028 clarksville pike
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923,6528 thunderbird dr
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652,2603 old matthews rd
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861,604 croley dr
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728,4484 lavergne couchville pike


(3) Identify addresses where multiple missed pickups occurred in a new dataframe

In [ ]:
## tester=trash_missed_pickups.groupby(['Address_lower','Trash Hauler'])
#tester2=tester.size().reset_index(name='num_trash_haulers')
#tester2.head()
#print(tester2['num_trash_haulers']>1)

In [14]:
total_missed_pickups=trash_missed_pickups['Address_lower'].value_counts().reset_index(name='total_count')
total_missed_pickups.head(),

(                                       Address_lower  total_count
 0  12546 old hickory blvd, nashville, tn 37013, u...           19
 1                           5135 hickory hollow pkwy           17
 2  802 crescent rd, nashville, tn 37205, united s...           17
 3  1816 jo johnston ave, nashville, tn 37203, uni...           16
 4   607 estes rd, nashville, tn 37215, united states           15,)

In [15]:
total_missed_pickups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11207 entries, 0 to 11206
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Address_lower  11207 non-null  object
 1   total_count    11207 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 175.2+ KB


In [16]:
mult_missed_pickups=total_missed_pickups[total_missed_pickups['total_count']>=2]

In [17]:
mult_missed_pickups.head()

,Address_lower,total_count
0,"12546 old hickory blvd, nashville, tn 37013, u...",19
1,5135 hickory hollow pkwy,17
2,"802 crescent rd, nashville, tn 37205, united s...",17
3,"1816 jo johnston ave, nashville, tn 37203, uni...",16
4,"607 estes rd, nashville, tn 37215, united states",15


(4)Calculate the total fine = total_fined_missed_pickups *200 and print the amount.

In [19]:
print(mult_missed_pickups['total_count'].sum())
total_fined_missed_pickups=mult_missed_pickups['total_count'].sum()-mult_missed_pickups['total_count'].count()
print('The total fine for missed pickups is $'+str(total_fined_missed_pickups*200)+'.')

6124
The total fine for missed pickups is $762600.


The total fine for the missed pickups is $762,600

(5) What are the other types of complaints?

In [22]:
trash_data_by_request=trash_pickup_data.Request.unique()

In [23]:
print(trash_data_by_request)

['Trash - Backdoor' 'Trash - Curbside/Alley Missed Pickup'
 'Trash Collection Complaint' 'Damage to Property']


In [24]:
trash_haulers=trash_pickup_data['Trash Hauler'].unique()

In [25]:
print(trash_haulers)


['RED RIVER' 'WASTE IND' 'METRO' nan 'Metro']


In [26]:
trash_data_by_route=trash_missed_pickups.groupby('Trash Route').count()

In [27]:
print(trash_data_by_route['Request Number'])

Trash Route
1201     160
1202      87
1202S      4
1203      68
1204      61
        ... 
9504      38
9505      46
9506      57
9507      61
9508     109
Name: Request Number, Length: 172, dtype: int64


In [28]:
trash_data_by_hauler=trash_missed_pickups.groupby('Trash Hauler').count()
print(trash_data_by_hauler['Request Number'])

Trash Hauler
METRO         2646
Metro           42
RED RIVER    10715
WASTE IND     1019
Name: Request Number, dtype: int64


In [29]:
metro_pct_of_complaints=round(2688/(2688+10715+1019.)*100,2)
print('Metro was responsible for '+str(metro_pct_of_complaints)+'% of the total missed pickup complaints')

Metro was responsible for 18.64% of the total missed pickup complaints
